## Part 1. Rates Scrape

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import pandas as pd
import os
import time
import docx
from docx.shared import Pt

In [52]:
time_today = time.strftime('%d_%b_%Y',time.localtime())

In [53]:
mydoc = docx.Document()

style = mydoc.styles['Normal']
font = style.font
font.name = 'Arial'
font.size = Pt(12)

In [54]:
rates_df = pd.read_csv('rates')
rates_df

,Unnamed: 0,name,rate
0,0,libor,2.86000
1,1,sofr,5.09366
2,2,sonia,4.93100
3,3,euribor,3.58500


In [55]:
prior_libor = rates_df['rate'].iloc[0]
prior_sofr = rates_df['rate'].iloc[1]
prior_sonia = rates_df['rate'].iloc[2]
prior_euribor = rates_df['rate'].iloc[3]

In [56]:
driver = webdriver.Chrome()
driver.get("https://www.nyse.com/bell/calendar")
time.sleep(3)

In [57]:
first_bell = driver.find_element(By.XPATH, '/html/body/div[1]/div/div[1]/div[3]/div[2]/div[1]/div[2]/div[1]/h3')

if 'Opening Bell' in first_bell.text:
    company_opening_bell = driver.find_element(By.XPATH, '/html/body/div[1]/div/div[1]/div[3]/div[2]/div[1]/div[2]/div[1]/h3').text
else:
    company_opening_bell = driver.find_element(By.XPATH, '/html/body/div[1]/div/div[1]/div[3]/div[2]/div[2]/div[2]/div[1]/h3').text

opening_company = company_opening_bell.split('Rings')[0][:-1]

print('NYSE Opening Bell: ' + opening_company)
first_par = mydoc.add_paragraph()
first_par.style = mydoc.styles['Normal']

first_par.add_run('NYSE Opening Bell: ').bold = True
first_par.add_run(opening_company)

NYSE Opening Bell: American Homes 4 Rent (NYSE: AMH)


In [58]:
driver.get('https://www.wsj.com/market-data/bonds')
time.sleep(5)

today_libor_oneyr = driver.find_element(By.XPATH, '//*[@id="root"]/div/div/div/div[2]/div[4]/div[1]/div[6]/table/tbody/tr[5]/td[2]').text
#last_libor_oneyr = driver.find_element(By.XPATH, '//*[@id="root"]/div/div/div/div[2]/div[4]/div[1]/div[6]/table/tbody/tr[5]/td[3]').text
#//*[@id="root"]/div/div/div/div[2]/div[4]/div[1]/div[6]/table/tbody/tr[5]/td[2]

libor_chg = round((float(today_libor_oneyr) - float(prior_libor))/float(prior_libor) * 100,5)

print('WSJ 1-Year LIBOR:' + str(today_libor_oneyr) + '%' + ' Change is:' + str(libor_chg) + '%')
first_par.add_run('\nWSJ 1-Year LIBOR: ').bold = True
first_par.add_run(str(today_libor_oneyr) + '%' + ' (' + str(libor_chg) + '%' + ' from previous rate'+')')

WSJ 1-Year LIBOR:2.86% Change is:0.0%


In [59]:
driver.get('https://www.newyorkfed.org/markets/reference-rates/sofr-averages-and-index')
time.sleep(3)

today_sofr_ninetyday = driver.find_element(By.XPATH,'//*[@id="pr_id_1-table"]/tbody/tr[1]/td[3]').text
#last_sofr_ninetyday = driver.find_element(By.XPATH, '//*[@id="pr_id_1-table"]/tbody/tr[2]/td[3]').text

sofr_chg = round((float(today_sofr_ninetyday) - float(prior_sofr))/float(prior_sofr) * 100,5)

print('Federal Reserve Bank of NY 90-day SOFR:' + str(today_sofr_ninetyday) + '%' + ' Change is:' + str(sofr_chg) + '%')
first_par.add_run('\nFederal Reserve Bank of NY 90-day SOFR: ').bold = True
first_par.add_run(str(today_sofr_ninetyday) + '%' + ' (' + str(sofr_chg) + '%' + ' from previous rate'+')')

Federal Reserve Bank of NY 90-day SOFR:5.09929% Change is:0.11053%


In [60]:
#1-month SONIA and chg
driver.get('https://www.global-rates.com/en/interest-rates/sonia/sonia.aspx')
time.sleep(3)

today_sonia_onemth = driver.find_element(By.XPATH,'//*[@id="ctl00"]/table[4]/tbody/tr/td/table/tbody/tr[2]/td[2]/table[4]/tbody/tr/td[1]/table/tbody/tr[2]/td[2]').text[:-1]
#last_sonia_onemth = driver.find_element(By.XPATH, '//*[@id="euribor-sonia-gilt-and-swap-rates"]/div/div[2]/div[2]/div/div/table/tbody/tr[2]/td[3]').text[:-1]

sonia_chg = round((float(today_sonia_onemth) - float(prior_sonia))/float(prior_sonia) * 100,5)

print('Bank of England 1-Month SONIA Rate:'  + str(today_sonia_onemth) + '%' + ' Change is:' + str(sonia_chg) + '%')
first_par.add_run('\nBank of England 1-Month SONIA Rate: ').bold = True
first_par.add_run(str(today_sonia_onemth) + '%' + ' (' + str(sonia_chg) + '%' + ' from previous rate'+')')

Bank of England 1-Month SONIA Rate:4.9315 % Change is:0.01014%


In [61]:
#1-month EURIBOR and chg
driver.get('https://www.chathamfinancial.com/technology/european-market-rates')
time.sleep(3)

today_euribor_onemth = driver.find_element(By.XPATH,'//*[@id="euribor-sonia-gilt-and-swap-rates"]/div/div[2]/div[1]/div/div/table/tbody/tr[1]/td[2]').text[:-1]

euribor_chg = round((float(today_euribor_onemth) - float(prior_euribor))/float(prior_euribor) * 100,5)

print('EMMI 1-Month Euribor Rate:' + str(today_euribor_onemth) + '%' + ' Change is:' + str(euribor_chg) + '%')
first_par.add_run('\nEMMI 1-Month Euribor Rate: ').bold = True
first_par.add_run(str(today_euribor_onemth) + '%' + ' (' + str(euribor_chg) + '%' + ' from previous rate'+')')

EMMI 1-Month Euribor Rate:3.57800% Change is:-0.19526%


In [62]:
rates_df['rate'] = [today_libor_oneyr,today_sofr_ninetyday,today_sonia_onemth,today_euribor_onemth]
os.remove('rates')
rates_df.to_csv('rates',index=False)

## Part 2. Stock Scrape

In [63]:
import yfinance as yfi

tickers = ['UMG.AS','WILD.TO','GFF','LOU.HM','TMO.L','EVD.DE','LYV','PLBY','FZT']
corp_names = ['Universal Stock','WildBrain Stock','Griffon Stock','DEAG Stock',
             'Time Out Stock','CTS Eventim Stock','Live Nation Stock','Playboy Group Stock','Falcon\'s Beyond SPAC Stock']
exchanges = ['ENXTAM','TSX','NYSE','HAM','LSE','XTRA','NYSE','NASDAQ','NYSE']
second_par = mydoc.add_paragraph()
second_par.style = mydoc.styles['Normal']


for i in range(len(tickers)):
    temp_current_price = yfi.Ticker(tickers[i]).info['currentPrice']
    temp_currency = yfi.Ticker(tickers[i]).info['financialCurrency']
    corp_name = corp_names[i]
    close_chg = yfi.Ticker(tickers[i]).info['currentPrice'] - yfi.Ticker(tickers[i]).info['previousClose']
    
    print(corp_name + ' ' + '(' + exchanges[i] + ': ' + tickers[i] + ')' + ': ' + temp_currency + ' '+ str(temp_current_price) + '(' + str(round(close_chg,2)) + ' from previous close)')
    second_par.add_run('\n' + corp_name).bold = True
    second_par.add_run(' ' + '(' + exchanges[i] + ': ' + tickers[i] + ')' + ': ' + temp_currency + ' '+ str(temp_current_price) + '(' + str(round(close_chg,2)) + ' from previous close)')
    

Universal Stock (ENXTAM: UMG.AS): EUR 23.46(0.13 from previous close)
WildBrain Stock (TSX: WILD.TO): CAD 1.72(-0.02 from previous close)
Griffon Stock (NYSE: GFF): USD 41.72(0.14 from previous close)
DEAG Stock (HAM: LOU.HM): EUR 6.0(-0.25 from previous close)
Time Out Stock (LSE: TMO.L): GBP 48.5(0.0 from previous close)
CTS Eventim Stock (XTRA: EVD.DE): EUR 59.8(-2.25 from previous close)
Live Nation Stock (NYSE: LYV): USD 87.75(-1.58 from previous close)
Playboy Group Stock (NASDAQ: PLBY): USD 1.93(0.08 from previous close)
Falcon's Beyond SPAC Stock (NYSE: FZT): USD 10.47(-0.06 from previous close)


In [64]:
mydoc.save('C:\\Users\\Username\\Desktop' + '\\' + time_today + '.docx')

## Part 3. News Run

In [6]:
import os
import os.path

news_df = pd.read_csv(r'C:\Users\Username\Downloads\news_run.csv').rename(columns = {'News Run':'Company Name'}).drop(8)
news_df.head(5)

,Unnamed: 0,Unnamed: 1,Company Name,Unnamed: 3,Unnamed: 4
0,x,#,Keyword / Company Searches,Website Link,Website Link
1,NaN,1),Advance Publications,Advance,https://www.advance.com/
2,NaN,2),Ambassador Theatre Group,Home - ATG - Ambassador Theatre Group,https://www.atg.co.uk/
3,NaN,3),Andrew Lloyd Webber,Home - Andrew Lloyd Webber,https://www.andrewlloydwebber.com/
4,NaN,4),Anthem Entertainment,Anthem Entertainment | Anthem Entertainment Gr...,https://www.anthementertainment.com/


In [66]:
#cleaning
news_df['Company Name'].iloc[1:75]

1         Advance Publications
2     Ambassador Theatre Group
3          Andrew Lloyd Webber
4         Anthem Entertainment
5       Authentic Brands Group
                ...           
71                       Virgo
72                     Vivendi
73                   WildBrain
74               Wolfgang Puck
75                 Yoo Capital
Name: Company Name, Length: 74, dtype: object

In [67]:
news_run_df = pd.DataFrame(columns = ['Company Name','News Title','News Link'])
news_run_df

,Company Name,News Title,News Link


In [68]:
from selenium.common.exceptions import NoSuchElementException

In [69]:
def search(name):
    driver.get('https://news.google.com/')
    article_element = driver.find_element(By.XPATH, '//*[@id="gb"]/div[2]/div[2]/div[2]/form/div[1]/div/div/div/div/div[1]/input[2]')
    
    article_element.click()
    article_element.clear()
    article_element.send_keys(name + ' when:1d')
    article_element.send_keys(Keys.RETURN)

def news_run(name):
    search(name)
    
    for i in range(0,10):
        test_xpath = '//*[@id="yDmH0d"]/c-wiz[' + str(i) + ']/div/div[2]/div[2]/div/main/c-wiz/div[1]/div[1]/div/article/h3/a'
        try:
            driver.find_element(By.XPATH, test_xpath)
        except NoSuchElementException:
            driver.refresh()
            print(f"No article found at index {i}")
        else:
            first_article_xpath = '//*[@id="yDmH0d"]/c-wiz[' + str(i)+ ']/div/div[2]/div[2]/div/main/c-wiz/div[1]/div[1]/div/article/h3/a'
            second_article_xpath = '//*[@id="yDmH0d"]/c-wiz[' + str(i)+ ']/div/div[2]/div[2]/div/main/c-wiz/div[1]/div[2]/div/article/h3/a'
            third_article_xpath = '//*[@id="yDmH0d"]/c-wiz[' + str(i)+ ']/div/div[2]/div[2]/div/main/c-wiz/div[1]/div[3]/div/article/h3/a'
            fourth_article_xpath = '//*[@id="yDmH0d"]/c-wiz[' + str(i)+ ']/div/div[2]/div[2]/div/main/c-wiz/div[1]/div[4]/div/article/h3/a'
            fifth_article_xpath = '//*[@id="yDmH0d"]/c-wiz[' + str(i)+ ']/div/div[2]/div[2]/div/main/c-wiz/div[1]/div[5]/div/article/h3/a'
            break
    else:
        return
    
    try:
        first_article_xpath
    except:
        news_run_df.loc[len(news_run_df)] = [name,'No News Today','No News Today']
        return


    article_xpaths = [first_article_xpath,second_article_xpath,third_article_xpath,fourth_article_xpath,fifth_article_xpath]
    
    for i in range(len(article_xpaths)):
        #some companies do not have five articles when searched, then we break after the try
        try:
            temp_title = driver.find_element(By.XPATH, article_xpaths[i]).text
            temp_link = driver.find_element(By.XPATH, article_xpaths[i]).get_attribute('href')

            new_row = pd.Series({'Company Name': name, 'News Title': temp_title, 'News Link': temp_link})
            news_run_df.loc[len(news_run_df)] = [name,temp_title,temp_link]
        except:
            return




In [70]:
news_df['Company Name'].iloc[1:75].apply(news_run)

No article found at index 0
No article found at index 0
No article found at index 0
No article found at index 0
No article found at index 0
No article found at index 0
No article found at index 0
No article found at index 0
No article found at index 0
No article found at index 0
No article found at index 1
No article found at index 2
No article found at index 3
No article found at index 4
No article found at index 5
No article found at index 6
No article found at index 7
No article found at index 8
No article found at index 9
No article found at index 0
No article found at index 0
No article found at index 0
No article found at index 0
No article found at index 0
No article found at index 1
No article found at index 2
No article found at index 3
No article found at index 4
No article found at index 5
No article found at index 6
No article found at index 7
No article found at index 8
No article found at index 9
No article found at index 0
No article found at index 0
No article found at 

1     None
2     None
3     None
4     None
5     None
      ... 
71    None
72    None
73    None
74    None
75    None
Name: Company Name, Length: 74, dtype: object

In [71]:
news_run_df.drop_duplicates(keep='first',inplace=True,subset=['News Link'])